In [1]:
%pip install opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_net = cv2.dnn.readNetFromCaffe('models/deploy.prototxt', 'models/gender_net.caffemodel')
age_net = cv2.dnn.readNetFromCaffe('models/deploy_age2.prototxt', 'models/age_net.caffemodel')

GENDER_LIST = ['M', 'F']
AGE_LIST = ['3', '5', '10', '20', '35', '40', '50', '65']

# Open the default camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera opened successfully.")

    while True:
        
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Cannot receive frame (stream end?). Exiting ...")
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        cv2.putText(frame, "Press Q or ESC to exit", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        for (x, y, w, h) in faces:
            face_img = frame[y:y + h, x:x + w].copy()
            blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
            
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            gender = GENDER_LIST[gender_preds[0].argmax()]

            age_net.setInput(blob)
            age_preds = age_net.forward()
            age = AGE_LIST[age_preds[0].argmax()]
            
            label = f"{gender}" + " | ~" + f"{age}"
            label_color = (255, 255, 255)

            if label == 'Male':
                color = (255, 0, 0)
            else:
                color = (0, 255, 0)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_color, 1)

        cv2.imshow('Gender Detection', frame)

        if cv2.waitKey(1) == ord('q') or cv2.waitKey(1) == 27:
            break

    # release camera
    cap.release()
    cv2.destroyAllWindows()


Camera opened successfully.
